In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
import matplotlib.pyplot as plt

In [2]:
url = 'https://www.mountainproject.com/route-finder?diffMaxaid=75260&diffMaxboulder=20050&diffMaxice=38500&diffMaxmixed=60000&diffMaxrock=6800&diffMinaid=70000&diffMinboulder=20000&diffMinice=30000&diffMinmixed=50000&diffMinrock=1600&is_sport_climb=1&is_top_rope=1&is_trad_climb=1&pitches=0&selectedIds=119375690&sort1=popularity%20desc&sort2=rating&stars=0&type=rock&viewAll=1#settings'

In [3]:
html = urllib.request.urlopen(url)
parse = BeautifulSoup(html, 'html.parser')

In [4]:
#Pulling all of the links from the website
links = []
for link in parse.find_all('a'):
    links.append(link.get('href'))

In [5]:
nlist = []
links = list(filter(None, links))

In [6]:
#Pulling all of the links for routes
for li in links:
    if li[0:38] == 'https://www.mountainproject.com/route/':
        nlist.append(li)

In [7]:
#Reducing to top 200
nlist = nlist[0:200]

In [8]:
directory = {'Name':[], 'Rating':[], 'Stars':[], 'First Ascent':[],'Type':[], 'Length (ft)':[], 'Pitches':[]}
for i in range(0,200):
    url = nlist[i]
    r = requests.get(url)
    bs = BeautifulSoup(r.text)
    txt = bs.find("div", {"class": "col-md-9 float-md-right mb-1"}).get_text()
    name = txt.split("\n\n\n")[5].strip()
    #print(name)
    rating = txt.split("\n\n\n")[6].strip().strip("\n").split(" ")[0]
    try:
        stars = txt.split("\n\n\n")[10].strip().split(" ")[1]
    except:
        stars = txt.split("\n\n\n")[9].strip().split(" ")[1]
    try:
        txt1 = bs.find("div", {"class": "col-lg-7 col-md-6"}).get_text()
    except:
        txt1 = bs.find("div", {"class": "col-lg-12"}).get_text()
    txt3 = txt1.split("\n\n\n")[2].strip()
    txt1 = txt1.split("\n\n\n")[1].strip()
    txt1 = txt1.split("\n\n")[1].strip()
    txt4 = txt3.split("\n\n")[1].strip()
    fa = txt4.split(",")[0]
    tp = txt1.split(",")[0]
    if len(txt1.split(",")) > 3:
        tp = tp + "," + txt1.split(",")[1]
        txt1=txt1[7:]
    if len(txt1.split(",")) > 1:
        length = txt1.split(",")[1].strip().split(" ")[0]    
    else:
        length = "-"
    if len(txt1.split(",")) > 2:
        pitches = txt1.split(",")[2].strip().strip(" ")[0]
        pitches
    else:
        pitches = 1
        
    directory['Name'].append(name)                                  
    directory['Rating'].append(rating)
    directory['Stars'].append(stars)
    directory['First Ascent'].append(fa) 
    directory['Type'].append(tp)
    directory['Length (ft)'].append(length)
    directory['Pitches'].append(pitches)

In [9]:
df = pd.DataFrame(directory)

In [10]:
#Light cleaning for 3 problematic rows
df.at[115, 'Type'] = 'Sport'
df.at[115, 'Length (ft)'] = '160'
df.at[115, 'Pitches'] = '3'

df.at[117, 'Type'] = 'Sport, TR'
df.at[117, 'Length (ft)'] = '40'
df.at[117, 'Pitches'] = '1'

df.at[151, 'Type'] = 'Sport'
df.at[151, 'Length (ft)'] = '300'
df.at[151, 'Pitches'] = '4'

In [11]:
df

,Name,Rating,Stars,First Ascent,Type,Length (ft),Pitches
0,Tachycardia,5.7,3.6,Matthew Selman,Sport,300,3
1,49,5.12a,3.7,Bill Boyle,Sport,80,1
2,Streak,5.11b,3.6,Boone Speed,Sport,105,1
3,Zoaster Toaster,5.11d,3.5,Bill Boyle,Sport,90,1
4,The Hitchhiker,5.11c,3.5,Darren Knezek,Sport,125,1
...,...,...,...,...,...,...,...
195,Whiskey Lounge,5.9,1.9,Jason Stevens,Sport,45,1
196,Take it for the Team,5.11d,2.6,unknown,Sport,65,1
197,Willy on a Stick,5.10d,2.5,Dan Snyder,Sport,110,1
198,Gimme Shelter,5.11a,2.8,Greg Miller,Sport,120,1


In [12]:
df.to_csv('Data.csv')